# Querying chpaf MoPo

In [4]:
import rdflib
import pandas as pd

# Set pandas to display wide columns
pd.set_option('max_colwidth', 1000)

g = rdflib.Graph()
g.parse("../examples/mopo.ttl")

<Graph identifier=Nad36c8bf67904ce39671fef46c146a88 (<class 'rdflib.graph.Graph'>)>

## Chapter 1

In [5]:
query_string = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dcterm: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?department ?office ?id ?title ?description ?answer WHERE {

    ?registration_activity paf:activityType chpaf:Registration.

    ?registration_entity prov:wasGeneratedBy ?registration_activity;
        schema:identifier ?id;
        schema:title ?title;
        schema:description ?description;
        chpaf:department ?department;
        chpaf:office ?office.

    ?proposal_creation_activity prov:wasInformedBy ?registration_activity.

    ?proposal_entity prov:wasGeneratedBy ?proposal_creation_activity;
        chpaf:mopoAnswer ?answer.

    ?proposal_activity prov:wasInformedBy ?proposal_creation_activity.

    OPTIONAL {
    
        ?decision_activity prov:wasInformedBy ?proposal_activity;
            prov:startedAtTime ?decision_time.
    
    }

    FILTER (!BOUND(?decision_time) || ?decision_time >= "2023-01-01"^^xsd:date)


    FILTER (lang(?title) = "de")
    FILTER (lang(?description) = "de")
    FILTER (lang(?answer) = "de")

} ORDER BY ?department ?office

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

,department,office,id,title,description,answer
0,https://ld.admin.ch/FCh,nan,19.4390,Anlaufstelle(n) in Sachen Administrativ- und Disziplinaruntersuchungen,"Der Bundesrat wird beauftragt, eine oder mehrere Stellen zu bezeichnen, welche über die erforderlichen Verfahrenskenntnisse bezüglich Administrativ- und Disziplinaruntersuchungen verfügen, sich über den aktuellen Wissensstand und die Rechtsprechung in diesem Bereich auf dem Laufenden halten und dadurch bei Bedarf anderen Einheiten des Bundes Rechtsauskünfte erteilen und diese beraten können. Er soll überdies dafür sorgen, dass sich die durchführenden Stellen bei formellen und rechtlichen Fragen systematischer an diese Beratungsstelle(n) wenden.",Die am 1. Januar 2022 in Kraft getretenen Weisungen des Bundesrates vom 18. August 2021 über Administrativ- und Disziplinaruntersuchungen (BBl 2021 1903) bezeichnen Beratungsstellen in Sachen Administrativ- und Disziplinaruntersuchungen und legen die Konsultations- und Dokumentationspflichten der für die Anordnung dieser Untersuchungen zuständigen Stellen fest. Diese Weisungen sind für die Departemente und die ihnen unterstellten Verwaltungseinheiten sowie die Bundeskanzlei verbindlich und werden in der Praxis umgesetzt.\r\nDer Bundesrat erachtet das Anliegen der Motion als erfüllt und beantragt deren Abschreibung.
1,https://ld.admin.ch/FCh,nan,20.3240,Covid-19. Bildung eines Zentrums für Innovation in der Bundesverwaltung (Public Innovation Hub),"Der Bundesrat wird beauftragt, einen Bericht zur Schaffung eines ""Public Innovation Hubs"", in Verbindung mit einer Innovationsstrategie für die Verwaltung, vorzulegen. Ziel des Hubs ist die Verknüpfung von ""Innovationsschaffern"" mit der Verwaltung, damit innovative Ideen geprüft und allenfalls rasch umgesetzt werden können. Der Bericht soll auf folgende Punkte eingehen: \r\n1. Trägerschaft: Der Hub ist interdepartemental auszugestalten. Die Interessen der Kantone werden berücksichtigt.\r\n2. Vernetzung: Der Hub ist als Schnittstelle zwischen Verwaltung auf der einen und Forschung und Tech-Unternehmen auf der anderen Seite zu konzipieren.\r\n3. Funktionsweise: Dank des Hubs sollen innovative Ideen rasch und unkompliziert getestet, wo nötig mit ""Pilotnormen"" temporär ermöglicht und im Erfolgsfall verwaltungsweit (und gegebenenfalls über alle föderalen Ebenen) verbreitet und implementiert werden können.",Postulatsbericht vom 23. September 2022 «Covid 19. Bildung eines Zentrums für Innovation in der Bundesverwaltung (Public Innovation Hub)». \r\nDer Bundesrat erachtet das Anliegen des Postulats als erfüllt und beantragt dessen Abschreibung
2,https://ld.admin.ch/FCh,nan,20.3280,Wissenschaftliches Potenzial für Krisenzeiten nutzen,"Der Bundesrat wird beauftragt, in einem Bericht zu prüfen, wie ein interdisziplinäres wissenschaftliches Netzwerk oder Kompetenzzentrum für Krisenlagen geschaffen werden kann. Der Bericht soll die entsprechenden Vor- und Nachteile, mögliche Umsetzungsoptionen sowie den gesetzgeberischen Handlungsbedarf aufzeigen.",Postulatsbericht vom 23. November 2022 «Wissenschaftliches Potenzial für Krisenzeiten nutzen». \r\nDer Bundesrat erachtet das Anliegen der Postulate als erfüllt und beantragt deren Abschreibung.
3,https://ld.admin.ch/FCh,nan,20.3363,"Strategie ""Digitale Schweiz"" nach dem Coronavirus","Der Bundesrat wird aufgefordert, die Strategie ""Digitale Schweiz"" aufgrund der während der Epidemie gesammelten Erfahrungen zu aktualisieren. Der Bericht soll die Risiken und Chancen der vermehrten Nutzung der digitalen Hilfsmittel im beruflichen und privaten Rahmen aufzeigen, insbesondere in den Bereichen Arbeit, Bildung, Gesundheit und Datenschutz.",Postulatsbericht vom 16. Dezember 2022 «Strategie «Digitale Schweiz 2023» – Aktualisierung nach dem Coronavirus».Der Bundesrat erachtet das Anliegen des Postulats als erfüllt und beantragt dessen Abschreibung.\r\n\r\n
4,https://ld.admin.ch/FCh,nan,20.3542,Ein Kompetenzzentrum für die Zeit nach Covid-19,"Der Bundesrat wi

## MoPo with Changed Registration

In [6]:
query_string = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dcterm: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?new ?old WHERE {

    ?new prov:wasInformedBy+ ?old;
        paf:activityType chpaf:Registration .
    
    ?old paf:activityType chpaf:Registration .

}

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

,new,old
0,https://politics.ld.admin.ch/curia/22.3506/2023/registration-activity,https://politics.ld.admin.ch/curia/22.3506/2022/registration-activity
1,https://politics.ld.admin.ch/curia/22.3507/2023/registration-activity,https://politics.ld.admin.ch/curia/22.3507/2022/registration-activity
2,https://politics.ld.admin.ch/curia/22.3508/2023/registration-activity,https://politics.ld.admin.ch/curia/22.3508/2022/registration-activity
3,https://politics.ld.admin.ch/curia/22.3509/2023/registration-activity,https://politics.ld.admin.ch/curia/22.3509/2022/registration-activity


## All Activities for a Certain Curia Number

In [7]:
query_string = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dcterm: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT ?id ?activity WHERE {

    BIND ("02.3532" as ?id)

    ?registration_entity schema:identifier ?id;
        prov:wasGeneratedBy ?registration_activity.

    ?activity prov:wasInformedBy* ?registration_activity;
        

}

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

,id,activity
0,02.3532,https://politics.ld.admin.ch/curia/02.3532/2021/registration-activity
1,02.3532,https://politics.ld.admin.ch/curia/02.3532/2021/information-creation-activity
2,02.3532,https://politics.ld.admin.ch/curia/02.3532/2021/information-activity
3,02.3532,https://politics.ld.admin.ch/curia/02.3532/2022/information-creation-activity
4,02.3532,https://politics.ld.admin.ch/curia/02.3532/2022/information-activity
5,02.3532,https://politics.ld.admin.ch/curia/02.3532/2023/information-creation-activity
6,02.3532,https://politics.ld.admin.ch/curia/02.3532/2023/information-activity


## Proposal after Information

In [8]:
query_string = """

PREFIX paf: <https://paf.link/>
PREFIX chpaf: <https://ch.paf.link/>
PREFIX prov: <http://www.w3.org/ns/prov#>
PREFIX schema: <http://schema.org/>
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX dcterm: <http://purl.org/dc/terms/>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>


SELECT * WHERE {

    ?proposal_activity paf:activityType chpaf:Proposal;
        prov:wasInformedBy/prov:wasInformedBy ?information_activity.

    ?information_activity paf:activityType chpaf:Information.
        

}

"""

qres = g.query(query_string)

df = pd.DataFrame(qres, columns=qres.vars)
df

,information_activity,proposal_activity
0,https://politics.ld.admin.ch/curia/00.3536/2021/information-activity,https://politics.ld.admin.ch/curia/00.3536/2022/proposal-activity
1,https://politics.ld.admin.ch/curia/06.3009/2022/information-activity,https://politics.ld.admin.ch/curia/06.3009/2023/proposal-activity
2,https://politics.ld.admin.ch/curia/08.3493/2021/information-activity,https://politics.ld.admin.ch/curia/08.3493/2022/proposal-activity
3,https://politics.ld.admin.ch/curia/09.3089/2022/information-activity,https://politics.ld.admin.ch/curia/09.3089/2023/proposal-activity
4,https://politics.ld.admin.ch/curia/09.3535/2022/information-activity,https://politics.ld.admin.ch/curia/09.3535/2023/proposal-activity
...,...,...
112,https://politics.ld.admin.ch/curia/20.3886/2022/information-activity,https://politics.ld.admin.ch/curia/20.3886/2023/proposal-activity
113,https://politics.ld.admin.ch/curia/20.3921/2022/information-activity,https://politics.ld.admin.ch/curia/20.3921/2023/proposal-activity
114,https://politics.ld.admin.ch/curia/20.3933/2022/information-activity,https://politics.ld.admin.ch/curia/20.3933/2023/proposal-activity
115,https://politics.ld.admin.ch/curia/20.4211/2022/information-activity,https://politics.ld.admin.ch/curia/20.4211/2023/proposal-activity
